In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/My Drive/Github/selfiespredict

/content/drive/My Drive/Github/selfiespredict


In [ ]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


In [4]:
! pip install -e .

Obtaining file:///content/drive/My%20Drive/Github/selfiespredict
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Attempting uninstall: selfiespredict
    Found existing installation: selfiespredict 0.0.post1.dev69+g8166cb4.d20211219
    Uninstalling selfiespredict-0.0.post1.dev69+g8166cb4.d20211219:
      Successfully uninstalled selfiespredict-0.0.post1.dev69+g8166cb4.d20211219
  Running setup.py develop for selfiespredict


In [5]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [6]:
!pip install faerun

In [7]:
!pip install mhfp

In [8]:
# hide
import pandas as pd
import tmap
from faerun import Faerun
from mhfp.encoder import MHFPEncoder
from rdkit.Chem import AllChem
import numpy as np
import tqdm

In [9]:
with open('./data/tokenized_data/SMILES/USPTO_480k/tgt-val.txt') as file:
    smiles = file.readlines()

In [10]:
smiles = [item.replace(" ", "").replace("\n", "") for item in smiles]

In [37]:
df = pd.DataFrame()
df["Smiles"] = smiles

In [ ]:
df.head()

,Smiles,categories
0,CC(C)=O.COc1ccc(S(=O)(=O)Cl)cc1.[I-].[Na+]\n,2.0
1,CC(C)(C)[O-].N#Cc1c(N)nc(Cl)c(C#N)c1-c1ccccc1....,2.0
2,C1CCOC1.CCCCCCC.CN1CCC(CCO)CC1.Cc1ccc(N2CCN(C(...,2.0
3,CC(C)(C)OC(=O)N1CCC(COC(=O)C2CCC3CN2C(=O)N3OS(...,2.0
4,CCC12CCC3C4CCC(=O)C=C4CCC3C1C(O)CC2=O.ClCCl.O....,2.0


In [39]:
df['categories'] = np.zeros(len(df))

In [12]:
from selfiespredict.helpers.errormetrics import*
def diff(first, second):
        second = set(second)
        return [item for item in first if item not in second]

In [33]:
SELFIES = topN_accuracy(1, "./results/predictions_best1_selfies_with_brackets.txt", "./data/tokenized_data/SELFIESwithBrackets/USPTO_480k/tgt-val.txt")
SMILES = topN_accuracy(1, "./run/SMILES/Test_PredictionsSMILESModeltopVal1.txt", "./data/tokenized_data/SMILES/USPTO_480k/tgt-val.txt")
all = list(np.linspace(1,30000, 30000).astype(int))
Both = (list(set(SMILES[1]).intersection(SELFIES[1])))
onlySMILES = diff(SMILES[1],SELFIES[1])
onlySELFIES = diff(SELFIES[1],SMILES[1])
union = set(SMILES[1]).union(SELFIES[1])
none = diff(all, union)
len(none)+len(Both)+len(onlySMILES)+len(onlySELFIES)

30000

In [41]:
none = [item-1 for item in none]
both = [item-1 for item in Both]
onlySMILES = [item-1 for item in onlySMILES]
onlySELFIES = [item-1 for item in onlySELFIES]

In [45]:
df['categories'] = np.zeros(len(df))
df.at[none,"categories"] = 1 
df.at[both,"categories"] = 2 
df.at[onlySMILES,"categories"] = 3 
df.at[onlySELFIES,"categories"] = 4

In [49]:
# The number of permutations used by the MinHashing algorithm
perm = 512

# Initializing the MHFP encoder with 512 permutations
enc = MHFPEncoder(perm)

# Create MHFP fingerprints from SMILES
# The fingerprint vectors have to be of the tm.VectorUint data type
fingerprints = [tmap.VectorUint(enc.encode(s)) for s in df["Smiles"]]

In [50]:
# Initialize the LSH Forest
lf = tmap.LSHForest(perm)

# Add the Fingerprints to the LSH Forest and index
lf.batch_add(fingerprints)
lf.index()

# Get the coordinates
x, y, s, t, _ = tmap.layout_from_lsh_forest(lf)

In [60]:
# Now plot the data
faerun = Faerun(view="front", coords=False)

legend_labels = [
        (1.0, "None"),
        (2.0, "Both"),
        (3.0, "Only SMILES"),
        (4.0, "Only SELFIES")
    ]

faerun.add_scatter(
    "predictionMap",
    {   "x": x, 
        "y": y, 
        "c": list(df.categories.values), 
        "labels": df["Smiles"]},
    colormap="tab10",
    point_scale=5,
    max_point_size=10,   
    has_legend=True,
    categorical=False,
    shader="sphere",
    legend_labels = legend_labels
)

faerun.add_tree("ESOL_Basic_tree", {"from": s, "to": t}, point_helper="predictionMap", color="#222222")

# Choose the "smiles" template to display structure on hover
faerun.plot('predictionMap', template="smiles", notebook_height=750)